In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
df = pd.read_csv(r'C:\Users\ankit\Desktop\Crop\crop_yield.csv')
df.info()

df = df.drop(columns=['Production'])    
df = df.dropna()   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19689 entries, 0 to 19688
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Crop             19689 non-null  object 
 1   Crop_Year        19689 non-null  int64  
 2   Season           19689 non-null  object 
 3   State            19689 non-null  object 
 4   Area             19689 non-null  float64
 5   Production       19689 non-null  int64  
 6   Annual_Rainfall  19689 non-null  float64
 7   Fertilizer       19689 non-null  float64
 8   Pesticide        19689 non-null  float64
 9   Yield            19689 non-null  float64
dtypes: float64(5), int64(2), object(3)
memory usage: 1.5+ MB


In [3]:
df = pd.get_dummies(df, columns=['Crop', 'Season', 'State'], drop_first=True)

X = df.drop(columns=['Yield'])
y = df['Yield']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [4]:

def build_base_model(input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        Dense(64, activation='relu'),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer=Adam(), loss='mse', metrics=['mae'])
    return model


def build_dropout_model(input_dim):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_dim,)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer=Adam(), loss='mse', metrics=['mae'])
    return model

In [ ]:

base_model = build_base_model(X_train.shape[1])
dropout_model = build_dropout_model(X_train.shape[1])

history_base = base_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)
history_dropout = dropout_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)


base_eval = base_model.evaluate(X_test, y_test, verbose=0)
dropout_eval = dropout_model.evaluate(X_test, y_test, verbose=0)

print(" — Loss:", base_eval[0], "MAE:", base_eval[1])
print("Dropout  — Loss:", dropout_eval[0], "MAE:", dropout_eval[1])

y_pred_base = base_model.predict(X_test)
y_pred_dropout = dropout_model.predict(X_test)


r2_base = r2_score(y_test, y_pred_base)
r2_dropout = r2_score(y_test, y_pred_dropout)

print(" — R²:", r2_base)
print("Dropout — R²:", r2_dropout)

c:\Users\ankit\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
